# handling dates and timestamp

In [1]:
import findspark
findspark.init('/home/aditya/spark-3.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("market").getOrCreate()

In [3]:
df = spark.read.csv('app_stock.csv',inferSchema=True, header=True)


In [4]:
df.show()

+----------+------+------+------+------+--------+---------+
|      Date|  Open|  High|   Low| Close|  Volume|Adj Close|
+----------+------+------+------+------+--------+---------+
|2012-03-30|608.77|610.56|597.94|599.55|26050900|   599.55|
|2012-03-29|612.78|616.56|607.23|609.86|21668300|   609.86|
|2012-03-28|618.38|621.45|610.31|617.62|23385200|   617.62|
|2012-03-27|606.18|616.28|606.06|614.48|21628200|   614.48|
|2012-03-26|599.79|607.15|595.26|606.98|21259900|   606.98|
|2012-03-23|600.49| 601.8| 594.4|596.05|15359900|   596.05|
|2012-03-22|597.78| 604.5|595.53|599.34|22281100|   599.34|
|2012-03-21|602.74|609.65|601.41| 602.5|22958200|    602.5|
|2012-03-20|599.51| 606.9|591.48|605.96|29166500|   605.96|
|2012-03-19|598.37|601.77|589.05| 601.1|32187000|    601.1|
|2012-03-16|584.72| 589.2| 578.0|585.57|29481700|   585.57|
|2012-03-15|599.61|600.01|578.55|585.56|41418500|   585.56|
|2012-03-14|578.05|594.72| 575.4|589.58|50570100|   589.58|
|2012-03-13|557.54|568.18|555.75| 568.1|

In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [6]:
df.select(['Date','Open']).show()

+----------+------+
|      Date|  Open|
+----------+------+
|2012-03-30|608.77|
|2012-03-29|612.78|
|2012-03-28|618.38|
|2012-03-27|606.18|
|2012-03-26|599.79|
|2012-03-23|600.49|
|2012-03-22|597.78|
|2012-03-21|602.74|
|2012-03-20|599.51|
|2012-03-19|598.37|
|2012-03-16|584.72|
|2012-03-15|599.61|
|2012-03-14|578.05|
|2012-03-13|557.54|
|2012-03-12|548.98|
|2012-03-09|544.21|
|2012-03-08|534.69|
|2012-03-07| 536.8|
|2012-03-06|523.66|
|2012-03-05|545.42|
+----------+------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import (dayofmonth, dayofyear, hour, month,
                                   year, weekofyear, format_number,date_format)

In [12]:
df.select(dayofmonth(df['Date']).alias('day_month')).show()

+---------+
|day_month|
+---------+
|       30|
|       29|
|       28|
|       27|
|       26|
|       23|
|       22|
|       21|
|       20|
|       19|
|       16|
|       15|
|       14|
|       13|
|       12|
|        9|
|        8|
|        7|
|        6|
|        5|
+---------+
only showing top 20 rows



In [14]:
df.select(hour(df['Date']).alias('hour')).show()

+----+
|hour|
+----+
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
|   0|
+----+
only showing top 20 rows



In [38]:
### getting average closing price by year
newdf = df.withColumn('year',year(df['Date']))
newdf = newdf.groupBy('year').agg({'Close':'mean'}).select(['year','avg(Close)'])
# newdf = df.withColumnRenamed('avg_close','avg(Close)')
newdf = newdf.withColumnRenamed('avg(Close)', 'avg_close')
newdf.show()

+----+------------------+
|year|         avg_close|
+----+------------------+
|1990| 37.56268774703557|
|2003|18.544761904761902|
|2007| 128.2739043824701|
|2006| 70.81063745019918|
|1997|17.965849802371523|
|1988| 41.54007905138338|
|1994| 34.08134920634923|
|2004|35.526944444444446|
|1991| 52.49553359683792|
|1996|24.917559055118087|
|1989| 41.65976190476188|
|1998| 30.56511904761903|
|1985| 20.19367588932804|
|2012|503.67983870967726|
|1987| 53.88968379446637|
|2009|146.81412698412706|
|1995| 40.54210317460315|
|2001|20.219112903225806|
|1992| 54.80338582677165|
|2005|   52.401746031746|
+----+------------------+
only showing top 20 rows



In [42]:
newdf.select(['year', format_number('avg_close',2).alias('avg_close')]).show()

+----+---------+
|year|avg_close|
+----+---------+
|1990|    37.56|
|2003|    18.54|
|2007|   128.27|
|2006|    70.81|
|1997|    17.97|
|1988|    41.54|
|1994|    34.08|
|2004|    35.53|
|1991|    52.50|
|1996|    24.92|
|1989|    41.66|
|1998|    30.57|
|1985|    20.19|
|2012|   503.68|
|1987|    53.89|
|2009|   146.81|
|1995|    40.54|
|2001|    20.22|
|1992|    54.80|
|2005|    52.40|
+----+---------+
only showing top 20 rows



In [43]:
newdf1 = newdf.select(['year', format_number('avg_close',2).alias('avg_close')]).collect()